In [ ]:
!pip install scikit-learn==0.22.1

In [ ]:
!pip show imbalanced-learn

In [ ]:
pip install kaggler

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import random
import glob

import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss

from kaggler.model import AutoLGB
from kaggler.preprocessing import LabelEncoder

from pprint import pprint

from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
import six
import sys
sys.modules['sklearn.externals.six'] = six
from sklearn.externals import six
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split


import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def days_to_age(x) :
    return int(((x)/365)+1)

def month_to_year(x) :
    return int(((x)/12)+1)

In [ ]:
PATH = './drive/MyDrive/Credit/data/'
train = pd.read_csv(PATH+'train.csv')
test = pd.read_csv(PATH+'test.csv')
# submit = pd.read_csv(PATH+'sample_submission.csv')

train.drop(['index', 'FLAG_MOBIL'], inplace=True, axis=1)
test.drop(['index', 'FLAG_MOBIL'], inplace=True, axis=1)

In [ ]:
# Error Data Preprocessing

train['family_size'][[5825, 16791, 21096, 18879, 16110]] = 3.0
train['family_size'][14900] = 4.0
train['family_size'][[10568]] = 1.0
train['gender'][[3603, 7078, 21930, 8576, 25641, 11828, 11121, 12984, 1393, 21770, 4443, 7942, 26008]] = 'F'
train['family_type'][[3064, 23249, 22262, 18305, 21027, 5825, 24754, 6683, 14287, 11308, 3890, 16110, 3138, 10279]] = 'Married'
train['reality'][[13036, 23381, 22828, 22133, 19893, 3138]] = 'Y'
train['email'][[19918, 18073, 19322]] = 1
train['family_type'][[26056, 10568]] = 'Widow'
train['edu_type'][26056] = 'Secondary / secondary special'
train['income_type'][[6743, 17278, 23227, 1964, 26159, 14356, 9793, 10357]] = 'Commercial associate'
train['income_type'][[5786, 6695]] = 'State servant'

In [ ]:
# iden column 추가
train['iden'] = train.apply(lambda x : "".join([str(x[i]) for i in ['DAYS_BIRTH', 'income_total', 'DAYS_EMPLOYED', 'family_type', 'gender']]), axis=1)
test['iden'] = test.apply(lambda x : "".join([str(x[i]) for i in ['DAYS_BIRTH', 'income_total', 'DAYS_EMPLOYED', 'family_type', 'gender']]), axis=1)

# child_num이 5보다 큰 데이터는 삭제 -> 추후에 적용 예정
train = train[train['child_num'] <= 5]
# 16654, 4512
train.drop([1561, 20920, 8576, 9196, 13105], inplace=True)
train.reset_index(drop=True, inplace=True)

In [ ]:
df = pd.concat([train.drop(['credit'], axis=1), test])
df.loc[df['DAYS_EMPLOYED']>0, 'DAYS_EMPLOYED'] = 0
df.loc[df['DAYS_EMPLOYED'] == 0, 'occyp_type'] = 'Unemployed'
df.fillna('Unknown', inplace=True)

for minus in ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'begin_month'] :
    df[minus] = -df[minus].astype(int)

df['family_size'] = df['family_size'].astype(int)
df['income_total'] = df['income_total'].astype(int)
df['life'] = (df['income_total']/df['family_size']).astype(int)

df['before_EMPLOYED']=df['DAYS_BIRTH']-df['DAYS_EMPLOYED']
df['before_EMPLOYED_month']=np.floor((df['before_EMPLOYED'])/30)-((np.floor((df['before_EMPLOYED'])/30)/12).astype(int)*12)
df['before_EMPLOYED_week']=np.floor((df['before_EMPLOYED'])/7)-((np.floor((df['before_EMPLOYED'])/7)/4).astype(int)*4)

df['DAYS_BIRTH_month']=np.floor((df['DAYS_BIRTH'])/30)-((np.floor((df['DAYS_BIRTH'])/30)/12).astype(int)*12)
df['DAYS_BIRTH_week']=np.floor((df['DAYS_BIRTH'])/7)-((np.floor((df['DAYS_BIRTH'])/7)/4).astype(int)*4)

df['DAYS_EMPLOYED_month']=np.floor((df['DAYS_EMPLOYED'])/30)-((np.floor((df['DAYS_EMPLOYED'])/30)/12).astype(int)*12)
df['DAYS_EMPLOYED_week']=np.floor((df['DAYS_EMPLOYED'])/7)-((np.floor((df['DAYS_EMPLOYED'])/7)/4).astype(int)*4)

df['gen_fam_type'] = df.apply(lambda x : str(x['gender'])+'_'+str(x['family_type']), axis=1)
df['real_car'] = df.apply(lambda x : str(x['reality'])+'_'+str(x['car']), axis=1)

df['fam_ts'] = df.apply(lambda x : str(x['family_type'])+'_'+str(x['family_size']), axis=1)
df['gen_income'] = df.apply(lambda x : str(x['gender'])+'_'+str(x['income_type']), axis=1)
df['gen_worktype'] = df.apply(lambda x : str(x['gender'])+'_'+str(x['occyp_type']), axis=1)
df['fc'] = df['family_size'] - df['child_num']

df['begin_year'] = df['begin_month']//12

df['iden_begin'] = df.apply(lambda x : str(x['iden'])+'_'+str(x['begin_year']), axis=1)
df['fam_house'] = df.apply(lambda x : str(x['house_type'])+'_'+str(x['family_size']), axis=1)
df['gen_work'] = df.apply(lambda x : str(x['gender'])+'_'+str(x['work_phone']), axis=1)
df['age_gen'] = df['DAYS_BIRTH'].apply(lambda x : '60' if days_to_age(x)//10*10 >= 60 else ( '10' if days_to_age(x)//10*10 < 20 else str(days_to_age(x)//10*10)))+ '_' + df['gender']
df['income_edu'] = df.apply(lambda x : str(x['edu_type'])+'_'+str(x['income_type']), axis=1)
df['age'] =  df['DAYS_BIRTH'].apply(lambda x : '60' if days_to_age(x)//10*10 >= 60 else ( '10' if days_to_age(x)//10*10 < 20 else str(days_to_age(x)//10*10)))

df.drop(['begin_year'], axis=1, inplace=True)

In [ ]:
edu_dict = {'Academic degree' : 4, # 학사 이상
            'Higher education' : 3, # 고등학교 졸업
            'Incomplete higher' : 2, # 고등학교 중퇴
           'Secondary / secondary special' : 1, # 중학교
            'Lower secondary' : 0} # 중학교 미만

df['edu_type'] = df['edu_type'].apply(lambda x: edu_dict[x])

ob_col = list(set([x for x in df.columns if df[x].dtype == 'object']))

lbe = LabelEncoder(min_obs=10)
df[ob_col] = lbe.fit_transform(df[ob_col])

In [ ]:
y = train['credit']
n_trn = train.shape[0]
X = df.iloc[:n_trn]
X_tst = df.iloc[n_trn:]

n_class = 3
n_fold = 5

cv = StratifiedKFold(n_splits=n_fold, random_state=42)

p = np.zeros((X.shape[0], n_class), dtype=float)
p_tst = np.zeros((X_tst.shape[0], n_class), dtype=float)

params={'bagging_freq': 1, 'verbosity': -1, 'seed': 42, 'num_threads': -1, 'feature_pre_filter': False,\
        'num_class': 3, 'objective': 'multiclass', 'metric': 'multi_logloss', 'boosting': 'gbdt',\
        'bagging_fraction': 0.7000000000000001, 'feature_fraction': 0.5, 'lambda_l1': 0,\
        'lambda_l2': 10, 'learning_rate': 0.04881102117105835, 'max_depth': -1,\
        'min_child_samples': 10, 'num_leaves': 255}
features = df.columns
n_best = 259
for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y)):
    train_data = lgb.Dataset(X[features].iloc[i_trn], label=y[i_trn])
    clf = lgb.train(params, train_data, n_best, verbose_eval=100)
    
    p[i_val] = clf.predict(X[features].iloc[i_val])
    p_tst += clf.predict(X_tst[features]) / n_fold

from sklearn.metrics import log_loss
print(f'CV Log Loss: {log_loss(y, p):f}')
submit = pd.read_csv(PATH+'sample_submission.csv')

submit[submit.columns[1:]] = p_tst
submit.to_csv('20210519_01.csv', index=False)
def get_ans(df) :
    df.drop(['index'], axis=1, inplace=True)
    temp = pd.Series(df.apply(lambda x : 0 if x['0'] == max(x) else ( 1 if x['1'] == max(x) else 2),  axis=1))
    display(temp.value_counts()/len(temp))

get_ans(submit)
submit['predict'] = submit.apply(lambda x : 0 if x['0'] == max(x) else ( 1 if x['1'] == max(x) else 2),  axis=1)

In [ ]:
CV Log Loss: 0.699465
2    0.8113
1    0.1437
0    0.0450

In [ ]:
clf.feature_name(), clf.feature_importance()


### 중복 데이터를 없애버림
* CV Log Loss: 0.754563  
2    0.9049  
1    0.0851  
0    0.0100

* real_car
    * CV Log Loss: 0.731970  
2    0.8478  
1    0.1215  
0    0.0307
>실제 점수 7.06  

-----------------


### 아무 전처리를 하지 않았을 때
* CV Log Loss: 0.729974  
2    0.9004  
1    0.0871  
0    0.0125  

* 추가 전처리
    * CV Log Loss: 0.703694  
2    0.8199  
1    0.1374  
0    0.0427
>실제 점수 7.04  

----------------





## 전처리 순서

1. DAYS_EMPLOYED>0인 데이터를 0으로 만들어준다.  
    1-1. 그 데이터의 occyp_type에 unemploed 부여  
    1-2. 남은 occyp_type Null 값 fillna()  
<br>
2. child_num이 family_num보다 큰 이상 데이터 삭제  
    2-1. child_num이 5초과 데이터 삭제  
<br>  

3. Gender와 AGE 변수를 합하여 age_gen 변수 생성  
    3-1. 'gender' 변수는 그대로 두는 것이 좋음  
    3-2. 'age' 추가는 고려, 안하는 게 나은 듯  
    3-3. family와 child의 차이인 fc도 고려만, 안하는게 좋은듯  
<br>
4. income_total을 family_size로 나눈 life 변수 생성  
    4-1. ['income_total'] normalize  
        * MinMax보다 Zscor가 좋음
        * 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'begin_month'은 나중에 고려  
    4-2. child 컬럼 삭제

5. before_EMPLOYED 컬럼 추가, month, week
6. real_car 추가 후, reality, car 삭제
    * email, phone은 의미있는 변수


예외
중복 데이터 처리